# 소프트 마진 분류

## 소프트 마진 (Soft Margin) 분류
- 어느 정도의 오류를 허용하면서 가급적 최대 마진이 되는 초평면을 찾는다
- 이와 같이 소프트 마진 분류를 수행하는 선형 SVM을 서포트 벡터 분류기(support vector classifier)라고 한다
- 잘못 분류되는 데이터가 있지만 초평면을 찾을 수 있음
- 과대적합을 방지하거나 줄일 수 있음

![](../img/SVM_softmargin.png)

### 하이퍼파라미터 C(cost)를 이용하여 허용할 오류의 수준을 결정
- 즉, “원래 데이터와 다른 클래스로 분류되는 경우를 얼마나 많이 허용할 것인가”를 조정하는 규제 값
-  매개변수 C는 이론적으로 소프트 마진 분류를 위해 도입된 여유 변수(slack variable) ξ 의 값에 적용되는 변수

#### C의 값이 높은 경우, 오류에 대해서 더 엄격하게 적용
- 마진이 작아진다.
- 오분류율이 낮아진다.
- 과대적합이 될 수 있다.
![](../img/SVM_softmargin_lowC.png)

#### C의 값이 낮은 경우, 오류에 대해서 덜 엄격하게 적용
- 마진이 커진다.
- 오분류율이 높아진다.
- 과소적합이 될 수 있다.

![](../img/SVM_softmargin_highC.png)

## 사이킷런으로 선형 SVM (소프트 마진) 분류 수행